In [1]:
import torch
from torch import nn

from einops import rearrange
from einops.layers.torch import Rearrange

# helpers

def pair(t):
    return t if isinstance(t, tuple) else (t, t)

def posemb_sincos_2d(h, w, dim, temperature: int = 10000, dtype = torch.float32):
    y, x = torch.meshgrid(torch.arange(h), torch.arange(w), indexing="ij")
    assert (dim % 4) == 0, "feature dimension must be multiple of 4 for sincos emb"
    omega = torch.arange(dim // 4) / (dim // 4 - 1)
    omega = 1.0 / (temperature ** omega)

    y = y.flatten()[:, None] * omega[None, :]
    x = x.flatten()[:, None] * omega[None, :]
    pe = torch.cat((x.sin(), x.cos(), y.sin(), y.cos()), dim=1)
    return pe.type(dtype)

# classes

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(dim),  # Normalization is done along the last dimension by default
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, dim),
        )
    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64):
        super().__init__()
        inner_dim = dim_head *  heads
        self.heads = heads
        self.scale = dim_head ** -0.5
        self.norm = nn.LayerNorm(dim)

        self.attend = nn.Softmax(dim = -1)

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)
        self.to_out = nn.Linear(inner_dim, dim, bias = False)

    def forward(self, x):
        x = self.norm(x)

        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale

        attn = self.attend(dots)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                Attention(dim, heads = heads, dim_head = dim_head),
                FeedForward(dim, mlp_dim)
            ]))
    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        return self.norm(x)

class SimpleViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, channels = 3, dim_head = 64):
        super().__init__()
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)

        assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size.'

        patch_dim = channels * patch_height * patch_width

        self.to_patch_embedding = nn.Sequential(
            Rearrange("b c (h p1) (w p2) -> b (h w) (p1 p2 c)", p1 = patch_height, p2 = patch_width),
            nn.LayerNorm(patch_dim),
            nn.Linear(patch_dim, dim),
            nn.LayerNorm(dim),
        )

        self.pos_embedding = posemb_sincos_2d(
            h = image_height // patch_height,
            w = image_width // patch_width,
            dim = dim,
        )

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim)

        self.pool = "mean"
        self.to_latent = nn.Identity()

        self.linear_head = nn.Linear(dim, num_classes)

    def forward(self, img):
        device = img.device

        x = self.to_patch_embedding(img)
        x += self.pos_embedding.to(device, dtype=x.dtype)

        x = self.transformer(x)
        x = x.mean(dim = 1)

        x = self.to_latent(x)
        return self.linear_head(x)

In [2]:
pip install einops


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm  # For progress bar
import os

# 1. Prepare CIFAR-10 Dataset (Full Dataset)
def get_cifar10_dataloader(batch_size=128):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),  # Resize to match Vision Transformer input size
        transforms.ToTensor(),         # Convert to tensor
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
    ])

    train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
    test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

# 2. Define the Vision Transformer Model
def create_vit_model():
    return SimpleViT(
        image_size=256,   # Input image size
        patch_size=32,    # Patch size for splitting the image
        num_classes=10,   # CIFAR-10 has 10 classes
        dim=1024,         # Embedding dimension
        depth=6,          # Number of transformer layers
        heads=16,         # Number of attention heads
        mlp_dim=2048      # Feed-forward hidden layer size
    )

# 3. Define the Training Function with Progress Bar
def train_model(model, train_loader, test_loader, device, epochs=10, lr=0.001, save_path="./best_model.pth"):
    criterion = nn.CrossEntropyLoss()  # Loss function
    optimizer = optim.Adam(model.parameters(), lr=lr)  # Optimizer
    best_accuracy = 0.0  # Track the best accuracy

    model.to(device)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        # Use tqdm for a progress bar
        progress_bar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{epochs}]")
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            preds = model(images)
            loss = criterion(preds, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Update running loss
            running_loss += loss.item()
            progress_bar.set_postfix({"Loss": f"{running_loss / len(train_loader):.4f}"})

        # Evaluate the model
        accuracy = evaluate_model(model, test_loader, device)

        # Save the best model
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), save_path)
            print(f"New best model saved with accuracy: {best_accuracy:.2f}%")

# 4. Define the Evaluation Function
def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            preds = model(images)
            _, predicted = torch.max(preds, 1)

            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy:.2f}%")
    return accuracy

# 5. Main Function
if __name__ == "__main__":
    # Hyperparameters
    batch_size = 64
    epochs = 10
    lr = 0.001
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    save_path = "./best_model.pth"

    # Prepare data
    train_loader, test_loader = get_cifar10_dataloader(batch_size)

    # Create model
    model = create_vit_model()

    # Check if a saved model exists and load it
    if os.path.exists(save_path):
        print("Loading existing model...")
        model.load_state_dict(torch.load(save_path))
        print("Model loaded successfully!")

    # Train model
    train_model(model, train_loader, test_loader, device, epochs, lr, save_path)


Files already downloaded and verified
Files already downloaded and verified


Epoch [1/10]: 100%|██████████| 782/782 [03:46<00:00,  3.46it/s, Loss=2.0733]


Accuracy: 26.34%
New best model saved with accuracy: 26.34%


Epoch [2/10]: 100%|██████████| 782/782 [03:48<00:00,  3.43it/s, Loss=1.9098]


Accuracy: 30.55%
New best model saved with accuracy: 30.55%


...

Accuracy: 23.26%


Epoch [9/10]: 100%|██████████| 782/782 [03:45<00:00,  3.46it/s, Loss=1.9870]


Accuracy: 23.25%


Epoch [10/10]: 100%|██████████| 782/782 [03:45<00:00,  3.46it/s, Loss=1.9722]


Accuracy: 25.86%


In [ ]:

# 3. Define the Training Function with Progress Bar
def train_model(model, train_loader, test_loader, device, epochs=10, lr=0.001, save_path="./best_model.pth"):
    criterion = nn.CrossEntropyLoss()  # Loss function
    optimizer = optim.Adam(model.parameters(), lr=lr)  # Optimizer
    best_accuracy = 0.0  # Track the best accuracy

    model.to(device)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        # Use tqdm for a progress bar
        progress_bar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{epochs}]")
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            preds = model(images)
            loss = criterion(preds, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Update running loss
            running_loss += loss.item()
            progress_bar.set_postfix({"Loss": f"{running_loss / len(train_loader):.4f}"})

        # Evaluate the model
        accuracy = evaluate_model(model, test_loader, device)

        # Save the best model
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), save_path)
            print(f"New best model saved with accuracy: {best_accuracy:.2f}%")

# 4. Define the Evaluation Function
def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            preds = model(images)
            _, predicted = torch.max(preds, 1)

            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy:.2f}%")
    return accuracy

# 5. Main Function
if __name__ == "__main__":
    # Hyperparameters
    batch_size = 64
    epochs = 20
    lr = 0.001
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    save_path = "./best_model.pth"

    # Prepare data
    train_loader, test_loader = get_cifar10_dataloader(batch_size)

    # Create model
    model = create_vit_model()

    # Check if a saved model exists and load it
    if os.path.exists(save_path):
        print("Loading existing model...")
        model.load_state_dict(torch.load(save_path))
        print("Model loaded successfully!")

    # Train model
    train_model(model, train_loader, test_loader, device, epochs, lr, save_path)


Files already downloaded and verified
Files already downloaded and verified


/tmp/ipykernel_193781/1959310182.py:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(save_path))


Loading existing model...
Model loaded successfully!


Epoch [1/20]: 100%|██████████| 782/782 [03:45<00:00,  3.46it/s, Loss=1.9967]


Accuracy: 25.01%
New best model saved with accuracy: 25.01%


...

Epoch [13/20]: 100%|██████████| 782/782 [03:45<00:00,  3.47it/s, Loss=1.9068]


Accuracy: 29.77%
New best model saved with accuracy: 29.77%


Epoch [14/20]: 100%|██████████| 782/782 [03:43<00:00,  3.50it/s, Loss=1.9143]


Accuracy: 28.89%


Epoch [15/20]:  52%|█████▏    | 409/782 [01:57<01:51,  3.35it/s, Loss=0.9943]